In [ ]:
# 打开文件
with open('scene1.ks', 'r', encoding='utf-8') as f:
    data = f.readlines()
text = data

In [ ]:
# 提取日文，保存到jp.text
jp = []
num = len(text)
for line in text:
    if line[-4:-1] == '[p]':
        jp.append(line)
with open('jp.text', 'w', encoding='utf-8') as f:
    for line in jp:
        f.write(line)

In [44]:
# 根据文本建立空value字典，保存到 jp-chs.json

import json

with open('jp.text', 'r', encoding='utf-8') as f:
    text_jp = f.readlines()
j_c = dict()
print('总共', num, '条')
for line in text_jp:
    j_c[line[:-4]] = ''

with open('jp-chs.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(j_c))



总共 2891 条


In [47]:
# 打开jp-chs.json文件检查
import json
with open('jp-chs.json', 'r', encoding='utf-8') as f:
    js = json.loads(f.read())
print(list(js.keys())[:10])
print(len(list(js.keys())))

['「ええぇ？ちょっとナオ、何を言ってるの？全然意味がわからないんだけど…」', '「そ、そうだよね…。ごめん、なんでもない。忘れてくれ」', '「えっ…？\u3000何なの一体？」', '「ちょっと…疲れてるんだ。最近、仕事が忙しいのもあって…」', '「ごめん………」', '「…わ、私の方こそ、その…きつい言い方しちゃって…」', '「あ、いや…。俺が変なこと言っちゃったから…。お、驚いた、よね？」', '「…う、うん…」', '昨夜、妻の理子とベットで交わした会話。', '「理子を…他の男に抱かせたい」']
2725


In [58]:
# 把字典翻译
import json
from tencentfanyi import translate as t

with open('jp-chs.json', 'r', encoding='utf-8') as f:
    data = json.loads(f.read())
cnt = 0
cnt_last = 0
failed_list = []
for key in data:
    try:
        if not data[key]:
            ans = t(key)
            data[key] = ans
        cnt+=1
        # FIXME
        # if cnt == 100:
        #     break
        if cnt - cnt_last == 10:
            cnt_last = cnt
            print(cnt, '/', 2725)
            with open('jp-chs.json', 'w', encoding='utf-8') as f:
                f.write(json.dumps(data))
    except Exception as e:
        print(e)
        failed_list.append(key)
else:
    with open('jp-chs.json', 'w', encoding='utf-8') as f:
        f.write(json.dumps(data))
    with open('failed.text', 'w', encoding='utf-8') as f:
        for line in failed_list:
            f.write(line)
    print('失败：', len(failed_list), '个！')


10 / 2725
20 / 2725
30 / 2725
40 / 2725
50 / 2725
60 / 2725
70 / 2725
80 / 2725
90 / 2725
100 / 2725
110 / 2725
120 / 2725
130 / 2725
140 / 2725
150 / 2725
160 / 2725
170 / 2725
180 / 2725
190 / 2725
200 / 2725
210 / 2725
220 / 2725
230 / 2725
240 / 2725
250 / 2725
260 / 2725
270/ 2725
280 / 2725
290 / 2725
300 / 2725
310 / 2725
320 / 2725
330 / 2725
340 / 2725
350 / 2725
360 / 2725
370 / 2725
380 / 2725
390 / 2725
400 / 2725
410 / 2725
420 / 2725
430 / 2725
440 / 2725
450 / 2725
460 / 2725
470 / 2725
480 / 2725
490 / 2725
500 / 2725
510 / 2725
520 / 2725
530 / 2725
540 / 2725
550 / 2725
560 / 2725
570 / 2725
580 / 2725
590 / 2725
600 / 2725
610 / 2725
620 / 2725
630 / 2725
640 / 2725
650 / 2725
660 / 2725
670 / 2725
680 / 2725
690 / 2725
700 / 2725
710 / 2725
720 / 2725
730 / 2725
740 / 2725
750 / 2725
760 / 2725
770 / 2725
780 / 2725
790 / 2725
800 / 2725
810 / 2725
820 / 2725
830 / 2725
840 / 2725
850 / 2725
860 / 2725
870 / 2725
880 / 2725
890 / 2725
900 / 2725
910 / 2725
920 / 272

In [61]:
# 构建对比
with open('jp-chs.json', 'r', encoding='utf-8') as f:
    data = json.loads(f.read())
cnt = 0
with open('contrast.json', 'w', encoding='utf-8') as f:
    for i in data:
        if data[i]:
            f.write(i+' '+data[i]+'\n')
        else:
            print(i, data[i])

ドクッ、ドクッ、ドクッ、ドクッ、ドクッ、ドクッ、ドクッ… 
ドクッ、ドクッ、ドクッ、ドクッ、ドクッ、ドクッ、ドクッ、ドクッ、ドクッ、ドクッ 


In [60]:
# 替换原文件
with open('jp-chs.json', 'r', encoding='utf-8') as f:
    jp_chs = json.loads(f.read())
with open('scene1.ks', 'r', encoding='utf-8') as f:
    data = f.readlines()
cnt = 0
for line in data:
    key = line[:-4]
    if key in jp_chs:
        if line[0] == '「':
            data[cnt] = '「' + jp_chs[key][1:-1] + '」[p]\n'
        else:
            data[cnt] = jp_chs[key][1:-1] + '[p]\n'
    cnt += 1

with open('scene1.ks.test', 'w', encoding='utf-8') as f:
    for line in data
        f.write(line)


0


In [62]:
a = '1234'
print(a[1:-1])

23
